# mT5-small on TPU for sinhala qa data💥🚀

**Evaluation results for SLQuAD wit mt5-small**

1 epochs - {'exact_match': 12.052117263843648, 'f1': 36.26088844185597}

3 epochs - {'exact_match': 30.510314875135723, 'f1': 57.400774595753866}

5 epochs - {'exact_match': 34.093376764386534, 'f1': 61.98354759551874}

---

**Parallel training**

3 epochs - {'exact_match': 18.241042345276874, 'f1': 44.31761156420535}

7 epochs - {'exact_match': 37.13355048859935, 'f1': 64.35452857472686}

9 epochs - {'exact_match': 39.413680781758956, 'f1': 66.16331104953571}

# Setting up the environment

In [ ]:
VERSION = "20211010"  #@param ["1.5" , "20211010", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  86711      0 --:--:-- --:--:-- --:--:-- 86711
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20211010 ...
Found existing installation: torch 1.10.0+cu111
     |████████████████████████████████| 57 kB 2.1 MB/s 
Uninstalling torch-1.10.0+cu111:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.290 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompati

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.3 MB/s 


Install transformers and the nlp package. Restart colab after this

In [ ]:
!git clone https://github.com/huggingface/transformers.git
!pip install ./transformers
!pip install -U nlp

Cloning into 'transformers'...
remote: Enumerating objects: 94773, done.
remote: Total 94773 (delta 0), reused 0 (delta 0), pack-reused 94773
Receiving objects: 100% (94773/94773), 80.05 MiB | 34.73 MiB/s, done.
Resolving deltas: 100% (68809/68809), done.
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.8 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 79.1 MB/s 
     |████████████████████████████████| 61 kB 529 kB/s 
     |███████████████████

## Load and process data

In [ ]:
import os 
os.environ['LD_LIBRARY_PATH']='/usr/local/lib'

!echo $LD_LIBRARY_PATH
!sudo ln -s /usr/local/lib/libmkl_intel_lp64.so /usr/local/lib/libmkl_intel_lp64.so.1
!sudo ln -s /usr/local/lib/libmkl_intel_thread.so /usr/local/lib/libmkl_intel_thread.so.1
!sudo ln -s /usr/local/lib/libmkl_core.so /usr/local/lib/libmkl_core.so.1

!ldconfig
!ldd /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch.so

/usr/local/lib
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

	linux-vdso.so.1 (0x00007ffc34adb000)
	/usr/lib/x86_64-linux-gnu/libtcmalloc.so.4 (0x00007fef0e85f000)
	libtorch_cpu.so => /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch_cpu.so (0x00007fef03f19000)
	libc10.so => /usr/local/lib/python3.7/dist-packages/torch/lib/libc10.so (0x00007fef03cbb000)
	libmkl_intel_lp64.so.1 => /usr/local/lib/libmkl_intel_lp64.so.1 (0x00007fef0318a000)
	libmkl_intel_thread.so.1 => /usr/local/lib/libmkl_intel_thread.so.1 (0x00007fef00caa000)
	libmkl_core.so.1 => /usr/local/lib/libmkl_core.so.1 (0x00007feefcb71000)
	libomp.so.5 => /usr/lib/x86_64-linux-gnu/libomp.so.5 (0x00007feefc8bc000)
	libpthread.so.0 => /lib/x86_64-linux-gnu/libpthread.so.0 (0x00007feefc69d000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007feefc2ff000)
	libdl.so.2 => /lib/x86_64-linux-gnu/libdl.so.2 (0x00007feefc0fb000)
	libstdc++.so.6 => /usr/lib/x8

In [ ]:
!pip install --upgrade torch

In [ ]:
import torch
import nlp
from transformers import T5Tokenizer

In [ ]:
# tokenizer = T5Tokenizer.from_pretrained('t5-base')

model = 'google/mt5-small'
tokenizer = T5Tokenizer.from_pretrained(model)

In [ ]:
# process the examples in input and target text format and the eos token at the end 
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (example['question_translated'], example['context_translated'])
    example['target_text'] = '%s </s>' % example['text_translated']
    return example

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=1024)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=32)

    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }
    

    return encodings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

train_dataset = pd.read_csv('/content/drive/My Drive/squad_si_train_df.csv')
test_dataset = pd.read_csv('/content/drive/My Drive/squad_si_val_df.csv')

In [ ]:
train_dataset.head()

,Unnamed: 0,Unnamed: 0.1,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,3811,8968,එක්සත් ජනපදයේ 1959 දී ටෙනසිහි සෙනෙට් සභික එස්ට...,කොන්ග්රස් විභාගයට නායකත්වය දුන්නේ කවුද,සෙනෙට් සභික එස්ටෙස් කෙෆාවර්,30,57
1,1,7970,17261,නිව් හෙවන්හි නාගරික සැලසුම්කරණයේ දීර් traditio...,විසිවන සියවසේ මැද භාගයේදී විශාල වශයෙන් විනාශ ව...,එල්ම් සිටි,443,453
2,2,2935,6972,ආර්ජන්ටිනාවේ ක්රියාකාරීන් ප්රවෘත්ති සාකච්ඡාවකට...,විකල්ප ගමනක් සඳහා සැලසුම් කළ මාර්ගය කුමක්ද,ඔබෙලිස්ක් සිට නගර ශාලාව දක්වා,446,475
3,3,8694,18679,කෘෂිකාර්මික විවිධත්වය ‘සැලසුම්ගත’ විවිධත්වය හෝ...,ශාකභක්ෂකයන් වල්පැලෑටි විශේෂ සහ රෝග කාරක,ආශ්රිත විවිධත්වය,307,323
4,4,4748,10879,1910 සිට හමුදාව ස්ථාවර පියාපත් සහිත ගුවන් යානා...,එක්සත් ජනපදය පළමු ලෝක යුද්ධයට සම්බන්ධ වූයේ කුම...,1917,308,312


In [ ]:
test_dataset.iloc[0]['context_translated']

'රික්තයක ගුණාත්මක භාවය පද්ධතියේ ඉතිරිව ඇති පදාර්ථ ප්රමාණයෙන් දැක්වේ එවිට උසස් තත්ත්වයේ රික්තයක් යනු ඉතා සුළු පදාර්ථයක් ඉතිරිව ඇති එකකි රික්තය මූලික වශයෙන් මනිනු ලබන්නේ එහි නිරපේක්ෂ පීඩනයෙනි නමුත් සම්පූර්ණ චරිත නිරූපණයකට උෂ්ණත්වය සහ රසායනික සංයුතිය වැනි තවත් පරාමිතීන් අවශ්ය වේ වැදගත්ම පරාමිතීන්ගෙන් එකක් වන්නේ අවශේෂ වායූන්ගේ මධ්යන්ය නිදහස් මාර්ගය එම්එෆ්පී වන අතර එයින් පෙන්නුම් කරන්නේ අණු එකිනෙක හා ගැටීම් අතර ගමන් කරන සාමාන්ය දුර ප්රමාණයයි වායු ity නත්වය අඩු වන විට එම්එෆ්පී වැඩි වන අතර කුටිය පොම්ප අභ්යවකාශ යානා හෝ වෙනත් වස්තූන් වලට වඩා එම්එෆ්පී දිගු වන විට තරල යාන්ත්ර විද්යාවේ අඛණ්ඩ උපකල්පන අදාළ නොවේ මෙම රික්ත තත්වය ඉහළ රික්තයක් ලෙස හැඳින්වෙන අතර මෙම තන්ත්රයේ තරල ප්රවාහ අධ්යයනය කිරීම අංශු වායු ගතිකය ලෙස හැඳින්වේ වායුගෝලීය පීඩනයේදී වාතයේ එම්එෆ්පී ඉතා කෙටි 70 එන්එම් නමුත් 100 එම්පීඒ  6997100000000000000 ♠ 1 × 10−3 ටෝර් කාමර උෂ්ණත්වයේ වාතයෙහි එම්එෆ්පී දළ වශයෙන් මිලිමීටර් 100 ක් වන අතර එය එදිනෙදා වස්තූන් වැනි අනුපිළිවෙලට රික්තක නල එම්එෆ්පී වෑන් රථ ප්රමාණයට වඩා විශාල වන විට ක්රූක්ස් විකිරණ මාන

In [ ]:
import json
columns = ['question_translated','text_translated','context_translated','answer_start','answer_end']

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset

train = Dataset.from_pandas(train_dataset.loc[0:,columns])
test = Dataset.from_pandas(test_dataset.loc[0:,columns])

In [ ]:
train[0]

{'answer_end': 57,
 'answer_start': 30,
 'context_translated': 'එක්සත් ජනපදයේ 1959 දී ටෙනසිහි සෙනෙට් සභික එස්ටෙස් කෙෆාවර්ගේ නායකත්වයෙන් යුත් කොන්ග්රස් මණ්ඩල නඩු විභාගවලින් එෆ්ඩී ඇන්ඩ් සී පනත සංශෝධනය කිරීම සඳහා තල්ලුවක් මතුවිය මෙම නඩු විභාගය වෙළඳ දැන්වීම් අපයෝජනය drugs ෂධවල සැක සහිත කාර්යක්ෂමතාව සහ ඊට වඩා වැඩි අවශ්යතාවයක් ඇතුළු පුළුල් පරාසයක ප්රතිපත්තිමය කරුණු ආවරණය කළේය කර්මාන්තය නියාමනය කිරීම නව නීති සම්පාදනය සඳහා වූ විකාශනය තාවකාලිකව පුළුල් විවාදයක් යටතේ සලකුණු කර ඇති අතර නව ඛේදවාචකයක් මතු වූ අතර එය වඩාත් පුළුල් නියාමනයක අවශ්යතාව අවධාරනය කළ අතර නව නීති සම්මත කිරීම සඳහා ගාමක බලවේගය සපයන ලදී නව නීති සම්පාදනය සඳහා වූ විකාශනය තාවකාලිකව පුළුල් විවාදයක් යටතේ සලකුණු කර ඇති අතර නව ඛේදවාචකයක් මතු වූ අතර එය වඩාත් පුළුල් නියාමනයක අවශ්යතාව අවධාරනය කළ අතර නව නීති සම්මත කිරීම සඳහා ගාමක බලවේගය සපයන ලදී',
 'question_translated': 'කොන්ග්රස් විභාගයට නායකත්වය දුන්නේ කවුද',
 'text_translated': 'සෙනෙට් සභික එස්ටෙස් කෙෆාවර්'}

In [ ]:
sample = {'answer_end': 23,
 'answer_start': 20,
 'context_translated': 'සුභ උදෑසනක්. මගේ නම සංඛ.',
 'question_translated': 'ඔහුගේ නම කුමක් ද?',
 'text_translated': 'සංඛ'}

In [ ]:
sample_feature = add_eos_to_examples(sample)

In [ ]:
sample_feature

{'answer_end': 23,
 'answer_start': 20,
 'context_translated': 'සුභ උදෑසනක්. මගේ නම සංඛ.',
 'input_text': 'question: ඔහුගේ නම කුමක් ද?  context: සුභ උදෑසනක්. මගේ නම සංඛ. </s>',
 'question_translated': 'ඔහුගේ නම කුමක් ද?',
 'target_text': 'සංඛ </s>',
 'text_translated': 'සංඛ'}

In [ ]:
# load train and validation split of squad
# train_dataset  = nlp.load_dataset('squad', split=nlp.Split.TRAIN)
# valid_dataset = nlp.load_dataset('squad', split=nlp.Split.VALIDATION)

# map add_eos_to_examples function to the dataset example wise 
train = train.map(add_eos_to_examples)
# map convert_to_features batch wise
train = train.map(convert_to_features, batched=True)

test = test.map(add_eos_to_examples, load_from_cache_file=False)
test = test.map(convert_to_features, batched=True, load_from_cache_file=False)


# set the tensor type and the columns which the dataset should return
columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
train.set_format(type='torch', columns=columns)
test.set_format(type='torch', columns=columns)

  0%|          | 0/8318 [00:00<?, ?ex/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2225: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: T

  0%|          | 0/921 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
train

Dataset({
    features: ['answer_end', 'answer_start', 'attention_mask', 'context_translated', 'input_ids', 'input_text', 'question_translated', 'target_attention_mask', 'target_ids', 'target_text', 'text_translated'],
    num_rows: 8318
})

In [ ]:
len(train), len(test)

(8318, 921)

In [ ]:
# cach the dataset, so we can load it directly for training

torch.save(train, 'train_data.pt')
torch.save(test, 'valid_data.pt')

In [ ]:
train[0]['input_ids']

tensor([ 7680,   267, 12182,  ...,     0,     0,     0])

## Write training script

In [ ]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import numpy as np
import torch

from transformers import T5ForConditionalGeneration, T5Tokenizer, EvalPrediction
from transformers import (
    HfArgumentParser,
    DataCollator,
    Trainer,
    TrainingArguments,
    set_seed,
)


logger = logging.getLogger(__name__)

# prepares lm_labels from target_ids, returns examples with keys as expected by the forward method
# this is necessacry because the trainer directly passes this dict as arguments to the model
# so make sure the keys match the parameter names of the forward method
@dataclass
class T2TDataCollator: #(DataCollator):
    # def __call__(self, features):
    def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
        
        """
        Take a list of samples from a Dataset and collate them into a batch.
        Returns:
            A dictionary of tensors
        """
        input_ids = torch.stack([example['input_ids'] for example in batch])
        lm_labels = torch.stack([example['target_ids'] for example in batch])
        lm_labels[lm_labels[:, :] == 0] = -100
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        decoder_attention_mask = torch.stack([example['target_attention_mask'] for example in batch])
        

        return {
            'input_ids': input_ids, 
            'attention_mask': attention_mask,
            'labels': lm_labels, 
            'decoder_attention_mask': decoder_attention_mask
        }


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )

@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """
    train_file_path: Optional[str] = field(
        default='train_data.pt',
        metadata={"help": "Path for cached train dataset"},
    )
    valid_file_path: Optional[str] = field(
        default='valid_data.pt',
        metadata={"help": "Path for cached valid dataset"},
    )
    max_len: Optional[int] = field(
        default=512,
        metadata={"help": "Max input length for the source text"},
    )
    target_max_len: Optional[int] = field(
        default=32,
        metadata={"help": "Max input length for the target text"},
    )


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

    # we will load the arguments from a json file, 
    #make sure you save the arguments in at ./args.json
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath('args.json'))

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed
    set_seed(training_args.seed)

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    tokenizer = T5Tokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    model = T5ForConditionalGeneration.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )

    # Get datasets
    print('loading data')
    train_dataset  = torch.load(data_args.train_file_path)
    valid_dataset = torch.load(data_args.valid_file_path)
    print('loading done')

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=T2TDataCollator(),
        # prediction_loss_only=True
    )

    # Training
    if training_args.do_train:
        trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_process_zero():
            tokenizer.save_pretrained(training_args.output_dir)

    # Evaluation
    results = {}
    if training_args.do_eval and training_args.local_rank in [-1, 0]:
        logger.info("*** Evaluate ***")

        eval_output = trainer.evaluate()

        output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(eval_output.keys()):
                logger.info("  %s = %s", key, str(eval_output[key]))
                writer.write("%s = %s\n" % (key, str(eval_output[key])))
    
        results.update(eval_output)
    
    return results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()

## Train

In [ ]:
import json

In [ ]:
args_dict = {
  "num_cores": 8,
  'training_script': 'train_t5_squad.py',
  "model_name_or_path": 'google/mt5-small',
  "max_len": 1024 ,
  "target_max_len": 32,
  "output_dir": './models/tpu',
  "overwrite_output_dir": True,
  # "per_gpu_train_batch_size": 2,
  "per_device_train_batch_size":1,
  # "per_gpu_eval_batch_size": 1,
  "per_device_eval_batch_size": 1,
  # "gradient_accumulation_steps": 4,
  "learning_rate": 5e-4,
  "tpu_num_cores": 8,
  "num_train_epochs": 9,
  "do_train": True,
  "prediction_loss_only":True,
  "save_strategy": "no",
  "load_best_model_at_end":True,
  "logging_steps":200
  # "debug": False
  # "logging_strategy":"epoch"
  # "remove_unused_columns":False
}

with open('args.json', 'w') as f:
  json.dump(args_dict, f)

In [ ]:
import torch_xla.distributed.xla_multiprocessing as xmp

In [ ]:
xmp.spawn(_mp_fn, args=(), nprocs=8, start_method='fork')

01/02/2022 07:33:14 - WARNING - __main__ -   Process rank: -1, device: xla:1, n_gpu: 0, distributed training: False, 16-bits training: False
01/02/2022 07:33:14 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=False,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,


loading data
loading done


01/02/2022 07:33:27 - WARNING - __main__ -   Process rank: -1, device: xla:0, n_gpu: 0, distributed training: False, 16-bits training: False
01/02/2022 07:33:27 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=False,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,


loading data
loading data
loading done
loading data
loading done
loading data
loading data
loading done
loading done
loading done
loading data
loading done
loading data
loading done


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1066: FutureWarning: `model_path` is deprecated and will be removed in a future version. Use `resume_from_checkpoint` instead.
  FutureWarning,
***** Running training *****
  Num examples = 8318
  Num Epochs = 9
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9360


...

Step,Training Loss
200,5.169100
400,2.523100
600,2.092400
800,1.883300
1000,1.734600
1200,1.533100
1400,1.488000
1600,1.311000
1800,1.361600
2000,1.303500


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1066: FutureWarning: `model_path` is deprecated and will be removed in a future version. Use `resume_from_checkpoint` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1066: FutureWarning: `model_path` is deprecated and will be removed in a future version. Use `resume_from_checkpoint` instead.
  FutureWarning,


Step,Training Loss
200,5.169100
400,2.523100
600,2.092400
800,1.883300
1000,1.734600
1200,1.533100
1400,1.488000
1600,1.311000
1800,1.361600
2000,1.303500


Step,Training Loss
200,5.169100
400,2.523100
600,2.092400
800,1.883300
1000,1.734600
1200,1.533100
1400,1.488000
1600,1.311000
1800,1.361600
2000,1.303500




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./models/tpu
Configuration saved in ./models/tpu/config.json
Model weights saved in ./models/tpu/pytorch_model.bin
tokenizer config file saved in ./models/tpu/tokenizer_config.json
Special tokens file saved in ./models/tpu/special_tokens_map.json
Copy vocab file to ./models/tpu/spiece.model


## Eval

In [ ]:
## SQuAD evaluation script. Modifed slightly for this Sinhala QA task

from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    # prediction_tokens = normalize_answer(prediction).split()
    # ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction) & Counter(ground_truth)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction)
    recall = 1.0 * num_same / len(ground_truth)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (prediction == ground_truth)


def metric_max_over_ground_truths(metric_fn, prediction, ground_truth):
    scores_for_ground_truths = []
    score = metric_fn(prediction, ground_truth)
    scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(gold_answers, predictions):
    f1 = exact_match = total = 0

    for ground_truths, prediction in zip(gold_answers, predictions):
      total += 1
      exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
      f1 += metric_max_over_ground_truths(
          f1_score, prediction, ground_truths)
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

import nlp
from transformers import T5ForConditionalGeneration, T5Tokenizer

from tqdm.auto import tqdm

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('models/tpu').to('cpu') # because its loaded on xla by default
tokenizer = T5Tokenizer.from_pretrained('models/tpu')

In [ ]:
valid_dataset = torch.load('valid_data.pt')
dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=1)

In [ ]:
answers = []
for batch in tqdm(dataloader):
  outs = model.generate(input_ids=batch['input_ids'], 
                        attention_mask=batch['attention_mask'],
                        max_length=32,
                        early_stopping=True)
  outs = [tokenizer.decode(ids) for ids in outs]
  answers.extend(outs)

  0%|          | 0/921 [00:00<?, ?it/s]

In [ ]:
predictions = []
references = []

for ref, pred in zip(valid_dataset['text_translated'], answers):
  pred = pred.replace("<pad>", "").replace("</s>", "").rstrip().lstrip()
  predictions.append(pred)
  references.append(ref)

In [ ]:
predictions[0], references[0]

('තරල ප්රවාහ අධ්යයනය', 'තරල ප්රවාහ අධ්යයනය')

In [ ]:
predictions[170], references[170]

('1966', '1966')

In [ ]:
evaluate(references, predictions)

{'exact_match': 39.413680781758956, 'f1': 66.16331104953571}